In [1]:
import re
import os
import numpy as np
import pandas as np
import nltk
from nltk.tokenize import word_tokenize
from numpy import genfromtxt

#%% Import libraries
import torch
import torchmetrics
from torch.utils.data import DataLoader, TensorDataset
import os
from glob import glob
from sklearn.model_selection import train_test_split

# Load training and test dataframes

In [ ]:
# Load sentences and labels
train = pd.read_csv('train_dataframe.csv')
test = pd.read_csv('test_dataframe.csv')

In [ ]:
train_sentences = train['Sentences'].tolist()
train_labels = train["Labels"].tolist()

test_sentences = test['Sentences'].tolist()
test_labels = test["Labels"].tolist()

## SVD (Singular Value decompostion)

# CNN

## Load data:

In [137]:
batch_size = 256
num_epochs = 100 
learning_rate = 0.01
weight_decay = 0.

In [118]:
# Load SVD-matrix from csv-file
A_k = genfromtxt('SVD.csv', delimiter=',')

In [148]:
A_k

array([[-6.16473675e+00, -4.02082074e-01, -1.52501388e-01, ...,
        -4.01377674e-02, -9.24744540e-03,  2.00122848e-02],
       [-4.61413986e+00, -3.23343196e-01,  8.82422583e-03, ...,
         4.69880154e-03,  1.84452465e-02, -1.13838840e-02],
       [-5.46674470e+00, -2.37277261e-01,  4.61381935e-01, ...,
         2.01689315e-02,  3.47052724e-02, -1.68292466e-02],
       ...,
       [-5.39011312e+00, -1.48593809e+00,  1.13009978e+00, ...,
        -2.06424785e-02,  6.46485565e-02, -1.12069831e-01],
       [-7.26047414e+00,  1.32972128e+00, -1.20220301e+00, ...,
        -1.34227097e-01,  1.34009804e-01, -5.79210589e-02],
       [-5.64561839e+00, -2.04792378e+00,  7.58238299e-01, ...,
        -2.92956351e-03,  5.25144745e-02,  3.60375463e-02]])

In [149]:
A_k.shape

(10731, 200)

In [138]:
# Split SVD-matrix (A_k) into training- and test data:
train_input, test_input, train_labels, test_labels = train_test_split(
    A_k,             # The SVD-reduced matrix
    labels_list,    # The labels
    test_size=0.2,   # Percentage of data to use for testing (e.g., 20%)
    random_state=42, # For reproducibility
)

In [139]:
# Convert inputs
train_input_tensor = torch.tensor(train_input, dtype=torch.float32)  # Shape: (num_train_sentences, 200)
test_input_tensor = torch.tensor(test_input, dtype=torch.float32)    # Shape: (num_test_sentences, 200)

# Convert labels
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)  # Shape: (num_train_sentences,)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)    # Shape: (num_test_sentences,)

In [151]:
# Reshape the tensors for CNN input (adding the channel dimension)
train_input_tensor = train_input_tensor.unsqueeze(1)  # Shape: (num_train_sentences, 1, 200)
test_input_tensor = test_input_tensor.unsqueeze(1)    # Shape: (num_test_sentences, 1, 200)

In [154]:
train_input_tensor.shape, test_input_tensor.shape

(torch.Size([8584, 1, 200]), torch.Size([2147, 1, 200]))

In [172]:
# Create datasets
train_dataset = TensorDataset(train_input_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_input_tensor, test_labels_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Shuffle training data
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   # No shuffle for testing

## Build the CNN

In [156]:
#%% Device
# Run on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [185]:
 #%% Neural network
net = torch.nn.Sequential(
    torch.nn.Conv1d(1, 16, kernel_size=3), # 200 - 3 + 1 = 198 
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.MaxPool1d(kernel_size=2),     # 198 // 2 = 99

    torch.nn.Conv1d(16, 16, kernel_size=3),  # 99 - 3 + 1 = 97
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.MaxPool1d(kernel_size=2),     # 97 // 2 = 48

    torch.nn.Flatten(),                    
    torch.nn.Linear(16*48, 254),
    torch.nn.ReLU(),
    torch.nn.Linear(254, 1),
    torch.nn.Sigmoid()       # Sigmoid activation for binary classification
            

).to(device)

In [ ]:
# %% Load trained network from file
# net.load_state_dict(torch.load('net.pt'))

In [182]:
#%% Loss and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [183]:
#%% Metrics
accuracy_metric = torchmetrics.classification.BinaryAccuracy().to(device)

## Train

In [ ]:
#%% Train
step = 0
for epoch in range(num_epochs):
    accuracy_metric.reset()
    for inputs, labels in train_loader:
        step += 1

        # Put data on GPU 
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # Compute loss and take gradient step
        prediction = net(inputs)
        prediction = prediction.squeeze(1)  # Shape: [batch_size]
        loss = loss_function(prediction, labels)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

        # Update accuracy metric
        accuracy_metric.update(prediction, labels)

    # Print accuracy for epoch            
    acc = accuracy_metric.compute()
    print(f'Training accuracy = {acc}')

Training accuracy = 0.5031453967094421
Training accuracy = 0.49743708968162537
Training accuracy = 0.48858341574668884
Training accuracy = 0.49580615758895874
Training accuracy = 0.5010484457015991
Training accuracy = 0.4939422309398651


In [ ]:
# %% Save the trained model
# torch.save(net.state_dict(), 'net.pt')